<a href="https://colab.research.google.com/github/s1290093/origin/blob/main/YOLO11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!nvcc --version

Tue Nov  5 04:27:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 46.4 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/ultralytics/ultralytics.git@main

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-m9gjmx2e
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-m9gjmx2e
  Resolved https://github.com/ultralytics/ultralytics.git to commit 603fa84774376b10d8783ffa9017f6b9b9b84861
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt
!wget -P {HOME}/weights -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11l.pt

!ls -lh {HOME}/weights

total 88M
-rw-r--r-- 1 root root 50M Oct  1 19:45 yolo11l.pt
-rw-r--r-- 1 root root 39M Oct  1 19:45 yolo11m.pt


In [22]:
!curl -L "https://universe.roboflow.com/ds/zwHk3RRjCj?key=yxQ6NgOLFw" > roboflow.zip
!mkdir -p /content/data
!unzip -q roboflow.zip -d /content/data
!rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1990      0 --:--:-- --:--:-- --:--:--  1986
100 29.2M  100 29.2M    0     0  7709k      0  0:00:03  0:00:03 --:--:-- 11.1M


In [24]:
# YAMLファイルの読み込みと書き換えに必要なPyYAMLライブラリをインストール
import yaml

# YAMLファイルを読み込んで修正
yaml_path = "/content/data/data.yaml"

with open(yaml_path, "r") as file:
    data = yaml.safe_load(file)

# パスの修正とクラス名の変更
data['train'] = '/content/data/train/images'
data['val'] = '/content/data/valid/images'
data['test'] = '/content/data/test/images'
data['names'] = ['ship']  # クラス名を'boat'から'ship'に変更

# 修正した内容を再度YAMLファイルに書き込み
with open(yaml_path, "w") as file:
    yaml.dump(data, file)

print("YAMLファイルが修正されました。")


YAMLファイルが修正されました。


# train

In [26]:
!yolo task=detect mode=train epochs=300 batch=16 plots=True model={HOME}/weights/yolo11m.pt data=/content/data/data.yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model={HOME}/weights/yolo11m.pt, data=/content/data/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=